In [3]:
import os
import csv
import requests
from dotenv import load_dotenv

load_dotenv()

KEYCLOAK_URL = os.getenv("KEYCLOAK_URL").rstrip("/")
REALM = os.getenv("KEYCLOAK_REALM")
CLIENT_ID = os.getenv("KEYCLOAK_CLIENT_ID")
USERNAME = os.getenv("KEYCLOAK_USERNAME")
PASSWORD = os.getenv("KEYCLOAK_PASSWORD")
API_URL = os.getenv("API_URL")

def get_access_token():
    token_url = f"{KEYCLOAK_URL}/realms/{REALM}/protocol/openid-connect/token"
    data = {
        "grant_type": "password",
        "client_id": CLIENT_ID,
        "username": USERNAME,
        "password": PASSWORD
    }
    response = requests.post(token_url, data=data)
    response.raise_for_status()
    return response.json()["access_token"]

def call_protected_api(token):
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(API_URL, headers=headers)
    response.raise_for_status()
    return response.json()

def filter_and_process_devices(data, prefix="CSRovere_Contatore"):
    devices = []
    for item in data.get("data", []):
        device = item.get("device", {})
        device_name = device.get("name", "")
        if device_name.startswith(prefix):
            feature = item.get("location", {}).get("feature", "")
            if feature.startswith("POINT("):
                try:
                    coords = feature.replace("POINT(", "").replace(")", "").split()
                    lat, lon = coords[0], coords[1]
                    print(f"{device_name} -> lat: {lat}, lon: {lon}")
                    devices.append({"name": device_name, "latitude": lat, "longitude": lon})
                except Exception as e:
                    print(f"Errore nel parsing per '{device_name}': {e}")
    return devices

def save_to_csv(devices, filename="devices.csv"):
    with open(filename, mode="w", newline="") as csvfile:
        fieldnames = ["name", "latitude", "longitude"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(devices)
    print(f"\n{len(devices)} dispositivi salvati nel file '{filename}'.")

if __name__ == "__main__":
    token = get_access_token()
    data = call_protected_api(token)
    devices = filter_and_process_devices(data)
    save_to_csv(devices)


CSRovere_ContatoreG2Mis_Lagrein2_Irritre -> lat: 46.252325, lon: 11.172134
CSRovere_ContatoreG2Mis_Feldi_irritre -> lat: 46.245002, lon: 11.164803
CSRovere_ContatoreG2Mis_Irritre_Settore_FeldiAlti -> lat: 46.246968, lon: 11.170983
CSRovere_ContatoreG2Mis_Irritre_Settore_Ronchi -> lat: 46.249691, lon: 11.163422
CSRovere_ContatoreG2Mis_Irritre_Settore_FeldiBassi -> lat: 46.24579, lon: 11.161498
CSRovere_ContatoreG2Mis_Irritre_Settore_Lagrain2 -> lat: 46.252323, lon: 11.1721
CSRovere_ContatoreG2Mis_Irritre_Settore_Feldi -> lat: 46.245108, lon: 11.164813
CSRovere_ContatoreG2Mis_Irritre_Settore_Righeli -> lat: 46.251625, lon: 11.179056

8 dispositivi salvati nel file 'devices.csv'.


In [5]:
import os
import csv
import requests
from dotenv import load_dotenv

load_dotenv()

KEYCLOAK_URL = os.getenv("KEYCLOAK_URL").rstrip("/")
REALM = os.getenv("KEYCLOAK_REALM")
CLIENT_ID = os.getenv("KEYCLOAK_CLIENT_ID")
USERNAME = os.getenv("KEYCLOAK_USERNAME")
PASSWORD = os.getenv("KEYCLOAK_PASSWORD")
METEO_API_URL = "https://irritre-sensor.azure.openiotlab.eu/api/device-locations"  # nuova API meteo

def get_access_token():
    token_url = f"{KEYCLOAK_URL}/realms/{REALM}/protocol/openid-connect/token"
    data = {
        "grant_type": "password",
        "client_id": CLIENT_ID,
        "username": USERNAME,
        "password": PASSWORD
    }
    resp = requests.post(token_url, data=data)
    resp.raise_for_status()
    return resp.json()["access_token"]

def call_meteo_api(token):
    headers = {"Authorization": f"Bearer {token}"}
    resp = requests.get(METEO_API_URL, headers=headers)
    resp.raise_for_status()
    return resp.json()

def extract_and_save_stations(data, filename="weather_stations.csv", prefix=None):
    stations = []
    for item in data.get("data", []):
        name = item.get("name") or item.get("station_name") or ""
        feature = item.get("location", {}).get("feature", "")
        if feature.startswith("POINT("):
            try:
                lat, lon = feature.replace("POINT(", "").replace(")", "").split()
                # se c'è un prefix lo applichiamo
                if prefix is None or name.startswith(prefix):
                    print(f"{name} -> lat: {lat}, lon: {lon}")
                    stations.append({"name": name, "latitude": lat, "longitude": lon})
            except Exception as e:
                print(f"Errore nel parsing per '{name}': {e}")

    with open(filename, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["name", "latitude", "longitude"])
        writer.writeheader()
        writer.writerows(stations)
    print(f"\n{len(stations)} stazioni salvate in '{filename}'.")

if __name__ == "__main__":
    token = get_access_token()
    meteo_data = call_meteo_api(token)
    extract_and_save_stations(meteo_data)


 -> lat: 46.252325, lon: 11.172134
 -> lat: 46.245002, lon: 11.164803
 -> lat: 46.245791, lon: 11.161646
 -> lat: 46.246968, lon: 11.170983
 -> lat: 46.249691, lon: 11.163422
 -> lat: 46.24579, lon: 11.161498
 -> lat: 46.252323, lon: 11.1721
 -> lat: 46.245108, lon: 11.164813
 -> lat: 46.251625, lon: 11.179056
 -> lat: 46.322496, lon: 11.091914
 -> lat: 46.247697, lon: 11.181276
 -> lat: 46.256852, lon: 11.162863
 -> lat: 46.31596, lon: 11.089941
 -> lat: 46.324905, lon: 11.091405
 -> lat: 46.325195, lon: 11.095244

15 stazioni salvate in 'weather_stations.csv'.


In [6]:
import csv

# Dati delle due stazioni meteo selezionate
stations = [
    {"name": "meteo_station_1", "latitude": 46.245791, "longitude": 11.161646},
    {"name": "meteo_station_2", "latitude": 46.251625, "longitude": 11.179056}
]

# Nome del file CSV di output
output_file = "meteo_stations.csv"

# Scrittura nel file CSV
with open(output_file, mode='w', newline='') as csvfile:
    fieldnames = ["name", "latitude", "longitude"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for station in stations:
        writer.writerow(station)

print(f"CSV salvato con successo in '{output_file}'")

CSV salvato con successo in 'meteo_stations.csv'
